## Importing The Important Libraries 

In [2]:
import cv2
import numpy as np
import time 

result = "Cartoonified.jpg"
img_name = "camera.jpg"
window_name = 'image'

## Capturing Photo Using OpenCV2

In [3]:
# Set the timer 
TIMER = int(3)
  
# Open the camera
cap = cv2.VideoCapture(0)
  
while True:

    prev = time.time()
    
    while TIMER >= 0:
        ret, img = cap.read()

        # Display the countdown using cv2.puttext method
        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(img, str(TIMER),
                    (275, 125), font,
                    3, (255, 127, 127),
                    3, cv2.LINE_AA)
        cv2.imshow('Say Cheese', img)
        cv2.waitKey(125)
        
        # current time
        cur = time.time()

        # Update 
        if cur-prev >= 1:
            prev = cur
            TIMER -=1
    
    else:
        cv2.destroyAllWindows()
        ret, img = cap.read()

        # Display the clicked frame for 2 Seconds
        cv2.imshow('a', img)
        cv2.waitKey(2000)
        cv2.imwrite(img_name, img)
        break
 
# close the camera
cap.release()
  
# close all the opened windows
cv2.destroyAllWindows()

### Function to Read File

In [4]:
def read_img(filename):
    img = cv2.imread(filename)
    return img

In [5]:
# Show the captured image 

img = read_img(f"{img_name}")
cv2.imshow(window_name,img)
cv2.waitKey(3000)
cv2.destroyAllWindows()

## Making a function to modify Edges 

In [6]:
def edge_mask(img, line_size, blur_value):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray_blur = cv2.medianBlur(gray, blur_value)
  edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
  return edges

In [7]:
line_size = 5
blur_value = 5
edges = edge_mask(img, line_size, blur_value)
cv2.imshow(window_name,edges)
cv2.waitKey(3000)
cv2.destroyAllWindows()

## Color Quantization Function To Add Cartoon Effect

In [8]:
def color_quantization(img,k):
    # Transform the Image
    data = np.float32(img).reshape((-1,3))

    # Determine the criteria
    criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER,20,0.001)

    #implimenting k mean
    ret , label, center = cv2.kmeans(data,k,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    result= center[label.flatten()]
    result = result.reshape(img.shape)

    return result


In [9]:
total_color = 15
img = color_quantization(img, total_color)

In [10]:
blurred = cv2.bilateralFilter(img, d=5, sigmaColor=100,sigmaSpace=100)

In [11]:
cartoon = cv2.bitwise_and(blurred, blurred, mask=edges)


In [12]:
cv2.imshow(window_name,cartoon)
cv2.waitKey(5000)
cv2.imwrite(result,cartoon)
cv2.destroyAllWindows()